<a href="https://colab.research.google.com/github/jinsusong/study-paper-review/blob/main/15_RoBERTa_ARobustly_Optimized_BERT_Pretraining_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RoBERTa: ARobustly Optimized BERT Pretraining Approach